In [1]:
import os
print(os.getcwd())
!module load 2023
!hostname

/gpfs/home1/cmeo/Varun/blockGPT
gcn96.local.snellius.surf.nl


In [2]:
import sys
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Add the project root to the Python path to allow importing 'nowcasting'
#sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))
print(os.getcwd())
from dataset_knmi import KNMITorchDataset




/gpfs/home1/cmeo/Varun/blockGPT


/home/cmeo/.conda/envs/blockgpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")


Using GPU: NVIDIA H100


In [4]:
!echo $SLURM_CPUS_ON_NODE
len(os.sched_getaffinity(0))

16

In [7]:
dataset_dir = '/projects/0/prjs0951/Varun/KNMI/5mins/train'

knmi_dataset = KNMITorchDataset(
    dataset_dir=dataset_dir,
    seq_len=9,
    raw_seq_len=9,
    sample_mode="sequent",
    stride=1,
    batch_size=16,  # Inner batch size for the KNMIDataloader
    layout="NTHW",
    img_size=128
)

print(f"Dataset length: {len(knmi_dataset)}")


Dataset length: 4437


In [ ]:
dataloader = knmi_dataset.get_torch_dataloader(outer_batch_size=64, num_workers=14)

all_data = []
for data in dataloader:
    all_data.append(data)
    print(data.shape)

print("Finished loading data. Concatenating...")
concatenated_data = torch.cat(all_data, dim=0)

print(f"Shape of the concatenated data: {concatenated_data.shape}")


